In [1]:
from tkinter import *

In [31]:


class A:
    def __init__(self):
        self.window = Tk()
        self.window.geometry('500x500')

        self.translation_lal = Label(self.window, text = "Translation").pack()
        #self.translation_x_lal = Label(self.window, text = "Translation").pack(side="left")
        self.translation_x = Entry(self.window).pack()
        self.translation_range = Entry(self.window).pack()



        button = Button(self.window, text="클릭", command = self.callback)
        button.pack()
        button1 = Button(self.window, text="종료", command = self.abcd)
        button1.pack()

        self.r=IntVar()
        self.r1 = Radiobutton(self.window, text="True",variable=self.r,value=1,command=self.check)
        self.r1.pack(anchor=W)
        self.r2 = Radiobutton(self.window, text="False", variable=self.r,value=2, command=self.check)
        self.r2.pack(anchor=W)

        self.window.mainloop()
    
    def check(self):
        rr1 = self.r.get()
        print(rr1)
        return rr1



    def callback(self):
        self.lbl.configure(text = "결과 값 = " + str(self.txt.get()))
        print(str(self.txt.get()))
        #self.window.destroy()

    def abcd(self):
        self.sum()
        self.window.destroy()
        

    def sum(self):
        print("dd")


A()

  


1
dd
<function A.check at 0x000001BA407B4E58>


히스토그램 & 밝기 조절

In [3]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
from tkinter import *

In [4]:
path = 'C:/Users/CYM/Desktop/DIP_image'
src_path = path+'/src/'
label_path = path+'/label/'

sample_image1 = 'C:/Users/CYM/Desktop/DIP_image/src/18-40-02-02 (SLA).jpg'
sample_image2 = 'C:/Users/CYM/Desktop/DIP_image/src/clahe.png'


In [11]:
'''
폴더에서 n번째 이미지를 가져올 수 있는 옵션을 사용자에게 줄까요?
다만 이거까지 주게되면 총 3개의 입력값을 받아야 해서 지져분해 보일것 같긴 해요.

'''
class A:    
    def __init__(self):
        self.window = Tk()
        self.window.geometry('500x500')
        aa=Label(self.window,text='주의! 미리보기 종료시, 반드시 아무키나 눌러주세요!').pack()
        aa=Label(self.window,text='ㅇㅇㅇㅇㅇㅇㅇㅇㅇ').pack()
        self.c = Entry(self.window)
        self.b = Entry(self.window)
        self.c.insert(0,"0")
        self.b.insert(0,"0")
        self.c.pack()
        self.b.pack()
        
        cl_bm_but = Button(self.window, text="미리보기",command=self.getText)
        cl_bm_but.pack()    # CLAHE_BM 실행 버튼

        finish_but = Button(self.window, text="실행/Tk종료", command=self.fin)
        finish_but.pack()

        self.window.mainloop()
    
    # 버튼으로 tkinter 종료
    def fin(self):
        self.window.destroy()
        
    # Entry값 가져오기
    def getText(self):
        k = float(self.c.get())
        w = int(self.b.get())
        self.CLAHE_BN(k,w)



    def CLAHE_BN(self, clip, br):
        img_list = os.listdir(src_path)  
        #src폴더 내에 n번째 이미지 불러오기
        set_img_n = cv2.imread(src_path + img_list[0])                  

        sample_image = cv2.cvtColor(set_img_n, cv2.COLOR_BGR2YUV)
        clahe = cv2.createCLAHE(clipLimit=clip, tileGridSize=(8,8))

        # CLAHE setting
        if clip != 0:
            sample_image[:,:,0] = clahe.apply(sample_image[:,:,0])

        # brightness setting
        sample_image[:,:,0] = cv2.add(sample_image[:,:,0], br)


        sample_image = cv2.cvtColor(sample_image, cv2.COLOR_YUV2BGR)

        
        cv2.namedWindow('imgView', cv2.WINDOW_NORMAL)
        cv2.imshow('imgView',sample_image)
        

        cv2.waitKey(0)
        cv2.destroyAllWindows()
        '''
         if cv2.getWindowProperty('imgView',cv2.WND_PROP_VISIBLE) <= 0:
            cv2.destroyAllWindows()
            exit(1)



        if not(cv2.cvGetWindowHandle('imgView')):
                    cv2.destroyAllWindows()
                    exit(1)



        '''



In [12]:
A()

무한 루프

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import random

In [ ]:
class Augment:
  global cols
  global rows
  global channels
  
  def __init__(self, img, mask):
    """
    args:
      img : 지금은 image 경로. 추후에는 directory로 변경할 수 있음.
      mask : 지금은 label 경로. 추후에는 directory로 변경할 수 있음.
    instances:
      self.img : OpenCV로 읽어온 image
      self.mask : OpenCV로 읽어온 label
      self.rows : image의 rows
      self.cols : image의 cols
      self.channels : image의 channels
      self.temp : image와 mask가 합쳐진 numpy array
    """
    self.img = cv2.imread(img, cv2.IMREAD_UNCHANGED)
    self.mask = cv2.imread(mask, cv2.IMREAD_UNCHANGED)
    self.rows = None
    self.cols = None
    self.channels = None
    self.temp = None

  def image_mask(self, img, mask, color=0): # image와 mask 비교 후 조건에 맞으면 합치기
    """
    image와 mask를 비교 후 조건에 맞으면 합쳐준다. (image와 mask에 동일한 변환을 주기 위해)
    args:
      img : OpenCV로 읽어온 image
      mask : OpenCV로 읽어온 label
      color : image를 변환할 color. 0 = 원래 이미지 color, 1 = grayscale, 2 = BGR [default=0]
    returns:
      temp : image와 mask가 합쳐진 numpy array
    """
    if img.shape[:2] != mask.shape[:2]: # image 와 mask의 rows, cols의 크기는 같아야함.
      print(f"Image and mask's shape[:2] must be equal. Your image.shape[:2] = {img.shape[:2]}, and mask.shape = {mask.shape[:2]}")
      return None
    # rows, cols 을 구함
    if img.ndim == 2:
        self.rows, self.cols = img.shape
        self.channels = 1
    elif img.ndim == 3:
        self.rows, self.cols, self.channels = img.shape
    else:
        print(f"Image's ndim must be 2 or 3. Your input image's ndim is {img.ndim}")
        return None

    # image 색 변환
    if color == 0: # 그대로
      pass
    elif color == 1: # grayscale
      if self.channels == 1:
        print('Input image is already grayscale')
        pass
      elif self.channels == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
      elif self.channels == 4:
        img = cv2.cvtColor(img, cv2.COLOR_BGRA2GRAY)
      else:
        print(f'Maybe there is problem. self.channels = {self.channels}')
      self.channels = 1
    elif color == 2: # BGR
      if self.channels == 1:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
      elif self.channels == 3:
        print('Input image is already BGR')
        pass
      elif self.channels == 4:
        img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
      else:
        print(f'Maybe there is problem. self.channels = {self.channels}')
      self.channels = 3
    else:
      print(f'Color option must be 0, 1 or 2. Your color option = {color}')


    # image와 mask의 channel이 다를 경우 (ex: image는 3채널 rgb인데 mask는 grayscale일 경우)
    if img.shape != mask.shape:
        if img.ndim == 2:
            img = img.reshape((img.shape[0], img.shape[1], 1))
        elif mask.ndim == 2:
            mask = mask.reshape((mask.shape[0], mask.shape[1], 1))
        else:
            print(f'Maybe there is problem. img.shape = {img.shape}, mask.shape = {mask.shape}')
            return None
    else:
        if img.ndim == 2: # 둘 다 channel이 1일 경우 dimension을 추가해주어야 concate 후 분리 가능
            img = img.reshape((img.shape[0], img.shape[1], 1))
            mask = mask.reshape((mask.shape[0], mask.shape[1], 1))

    temp = np.concatenate((img, mask), axis=-1) # img와 mask를 합친 후 한 번에 수행
    self.temp = temp
    return temp

  def translation(self, temp, x_range, y_range, control=True):
    """
    x축은 -x_range ~ x_range, y축은 -y_range ~ y_range의 범위에서 랜덤한 값으로 이미지를 이동시켜준다.
    args:
      temp : image와 mask가 합쳐진 numpy array
      x_range : x축 방향으로 이동시킬 범위
      y_range : y축 방향으로 이동시킬 범위
      control : translation을 진행할 것인지에 대한 boolean 값. True면 진행 False면 진행 X [default=True]
    returns:
      ret_img : 변환된 image
      ret_mask : 변환된 mask
    """
    if control:
      x_range = np.abs(x_range)
      y_range = np.abs(y_range)
      x = random.randint(-x_range, x_range) # 지정한 범위 내 random한 값
      y = random.randint(-y_range, y_range) # 지정한 범위 내 random한 값

      # translation matrix
      M = np.float32([[1,0,x],
                      [0,1,y]])
      ret = cv2.warpAffine(temp, M, (self.cols, self.rows)) # translation
      ret_img = ret[:,:,:self.channels] # 변환된 img
      ret_mask = ret[:,:,self.channels:] # 변환된 mask
      return ret_img, ret_mask

In [3]:
from tkinter import *

ka = 1

def quit():
    global root
    lal.configure(text = "결과 값 = "+str(ka))
    
    root.quit()

root = Tk()
Button(root, text="Quit", command=quit).pack()
lal = Label(root, text="ㅇㅇㅇ")
lal.pack()

while True:
    ka+=1
    root.mainloop()
    #do something

KeyboardInterrupt: 